# 03 - web scraping and data transformations

1. [The TSA posts passenger numbers](https://www.tsa.gov/coronavirus/passenger-throughput) in a table but there is no download or API option. We can use BeautifulSoup to parse this table.
1. Transform the TSA passenger data in two ways to create two different charts
1. Create two charts inside this notebook with [Matplotlib](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)

__Library reference__
- [BeautifulSoup]()
- [pandas]()
- [Matplot for pandas]()
- [Datetime format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)

1. Turn the TSA's html table into a dataframe
    1. Create a list of column names
    1. Create a 2d array of data
    1. Format the data into two columns: date and value
1. Transform the data in two different ways for new different charts
1. Create two charts

In [147]:
# !pipenv uninstall matplotlib

In [148]:
#### Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import requests

In [149]:
# set display format for numbers (in what way?)

In [150]:
#!pip install html5lib

## 1. Turn the TSA's html table into a dataframe

In [151]:
# get html from from page
tsa_r = requests.get("https://www.tsa.gov/coronavirus/passenger-throughput")
tsa_r

<Response [200]>

In [152]:
# create a beautifulsoup object
doc = BeautifulSoup(tsa_r.text, 'html.parser')

#### table tag
![table selected](../answers/assets/table.png)

### a. Create a list column names

In [155]:
# turn thead into a column list
thead = doc.select('thead')
thead

[<thead>
 <tr>
 <th class="views-align-center views-field views-field-field-today-date views-align-center" id="view-field-today-date-table-column" scope="col">Date</th>
 <th class="views-align-center views-field views-field-field-2021-throughput views-align-center" id="view-field-2021-throughput-table-column" scope="col">2021 Traveler Throughput </th>
 <th class="views-align-center views-field views-field-field-2020-throughput views-align-center" id="view-field-2020-throughput-table-column" scope="col">2020 Traveler Throughput </th>
 <th class="views-align-center views-field views-field-field-2019-throughput views-align-center" id="view-field-2019-throughput-table-column" scope="col">2019 Traveler Throughput </th>
 </tr>
 </thead>]

In [162]:
# then find all th elements (because there is only 1 row)
ths = doc.select('th')
ths

[<th class="views-align-center views-field views-field-field-today-date views-align-center" id="view-field-today-date-table-column" scope="col">Date</th>,
 <th class="views-align-center views-field views-field-field-2021-throughput views-align-center" id="view-field-2021-throughput-table-column" scope="col">2021 Traveler Throughput </th>,
 <th class="views-align-center views-field views-field-field-2020-throughput views-align-center" id="view-field-2020-throughput-table-column" scope="col">2020 Traveler Throughput </th>,
 <th class="views-align-center views-field views-field-field-2019-throughput views-align-center" id="view-field-2019-throughput-table-column" scope="col">2019 Traveler Throughput </th>]

In [163]:
# and loop through each th to extract the text for a list

tsa_col = []

for th in ths:
    tsa_col.append(th.text.strip())

In [164]:
# print the list
tsa_col

['Date',
 '2021 Traveler Throughput',
 '2020 Traveler Throughput',
 '2019 Traveler Throughput']

### b. Create a 2d array of data
![tbody example](../answers/assets/tbody.png)

In [165]:
# turn data into an array of arrays (2d array)

tbody = doc.select('tbody')
tbody

[<tbody>
 <tr>
 <td class="views-field views-field-field-today-date views-align-center" headers="view-field-today-date-table-column">7/5/2021          </td>
 <td class="views-field views-field-field-2021-throughput views-align-center" headers="view-field-2021-throughput-table-column">2,160,147          </td>
 <td class="views-field views-field-field-2020-throughput views-align-center" headers="view-field-2020-throughput-table-column">755,555          </td>
 <td class="views-field views-field-field-2019-throughput views-align-center" headers="view-field-2019-throughput-table-column">2,748,718          </td>
 </tr>
 <tr>
 <td class="views-field views-field-field-today-date views-align-center" headers="view-field-today-date-table-column">7/4/2021          </td>
 <td class="views-field views-field-field-2021-throughput views-align-center" headers="view-field-2021-throughput-table-column">1,681,896          </td>
 <td class="views-field views-field-field-2020-throughput views-align-center" 

In [166]:
# turn tr tags into a list
trs = doc.select('tr')
trs

[<tr>
 <th class="views-align-center views-field views-field-field-today-date views-align-center" id="view-field-today-date-table-column" scope="col">Date</th>
 <th class="views-align-center views-field views-field-field-2021-throughput views-align-center" id="view-field-2021-throughput-table-column" scope="col">2021 Traveler Throughput </th>
 <th class="views-align-center views-field views-field-field-2020-throughput views-align-center" id="view-field-2020-throughput-table-column" scope="col">2020 Traveler Throughput </th>
 <th class="views-align-center views-field views-field-field-2019-throughput views-align-center" id="view-field-2019-throughput-table-column" scope="col">2019 Traveler Throughput </th>
 </tr>,
 <tr>
 <td class="views-field views-field-field-today-date views-align-center" headers="view-field-today-date-table-column">7/5/2021          </td>
 <td class="views-field views-field-field-2021-throughput views-align-center" headers="view-field-2021-throughput-table-column">2

In [167]:
# create a list of td tags inside each tr list

tr_list = []

for tr in trs:
    tds = tr.select('td')
    td_list = []
    for td in tds:
        td_list.append(td.text.strip())
    tr_list.append(td_list)

    
del tr_list[0]


In [168]:
# Check the length of the list and the first couple of items

len(tr_list)
tr_list[0:5]

#['2,160,147', '755,555', '2,748,718']

[['7/5/2021', '2,160,147', '755,555', '2,748,718'],
 ['7/4/2021', '1,681,896', '732,123', '2,795,369'],
 ['7/3/2021', '1,915,017', '466,669', '2,345,846'],
 ['7/2/2021', '2,196,411', '718,988', '2,184,253'],
 ['7/1/2021', '2,147,090', '764,761', '2,088,760']]

In [169]:
[type(item[0]) for item in tr_list]
[type(item[1]) for item in tr_list]
[type(item[2]) for item in tr_list]

[str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str

### c. Format the data into two columns: date and value

In [170]:
# create a function that will generate dates of preceding years
def format_date(d, column_year):
    # turn string into date object so we can perform datetime calculations on it
    d = datetime.date(datetime.strptime(d, '%m/%d/%Y'))

    # how many years past
    year_delta = d.year - column_year

    # you can apply addition and subtraction to get x-number of dates from/since now
    d = d - timedelta(weeks=52*year_delta)

    return d

In [171]:
# [0] = 1st row [1:...] = second column+

print(tr_list[0][1:])

['2,160,147', '755,555', '2,748,718']


In [192]:
# this double loop can be combined with the loop above that generates tr_list
# but i want to separate text extraction from formatting
passengers_per_day = []
# for each tr
for td_list in tr_list:
    # we need to find dates for 2020 and 2019 and align them with the html table format
    # turn string into date object so we can perform datetime calculations on it
    date_2021 = datetime.date(datetime.strptime(td_list[0], '%m/%d/%Y'))
    # the date for 2020 will be 52 weeks before 
    date_2020 = date_2021 - timedelta(weeks=52)
    # the date for 2019 will be 104 weeks before
    date_2019 = date_2021 - timedelta(weeks=104)
    # because the above is a repeatable process, how can move this to a function?
    
    date_list = [format_date(td_list[0], 2021),
                 format_date(td_list[0], 2020),
                 format_date(td_list[0], 2019)]
    
    # for each passenger column td_list[1:]
    for (index, td) in enumerate(td_list[1:]):
        # Create a new dictionary to populate with formatted date
        # index being the column that corresponds to the order of dates in the date_list above
        daily_passengers = {
            'date': date_list[index],
            'value': td.replace(',', '')
        }
        
        # if value does exist, change it to an integer (or else there will be an error on missing values)
        if(daily_passengers['value'] != ''):
            daily_passengers['value'] = int(daily_passengers['value'])
            
            # add each newly created dictionary to passengers_per_day list
            passengers_per_day.append(daily_passengers)

In [193]:
passengers_per_day

[{'date': datetime.date(2021, 7, 5), 'value': 2160147},
 {'date': datetime.date(2020, 7, 6), 'value': 755555},
 {'date': datetime.date(2019, 7, 8), 'value': 2748718},
 {'date': datetime.date(2021, 7, 4), 'value': 1681896},
 {'date': datetime.date(2020, 7, 5), 'value': 732123},
 {'date': datetime.date(2019, 7, 7), 'value': 2795369},
 {'date': datetime.date(2021, 7, 3), 'value': 1915017},
 {'date': datetime.date(2020, 7, 4), 'value': 466669},
 {'date': datetime.date(2019, 7, 6), 'value': 2345846},
 {'date': datetime.date(2021, 7, 2), 'value': 2196411},
 {'date': datetime.date(2020, 7, 3), 'value': 718988},
 {'date': datetime.date(2019, 7, 5), 'value': 2184253},
 {'date': datetime.date(2021, 7, 1), 'value': 2147090},
 {'date': datetime.date(2020, 7, 2), 'value': 764761},
 {'date': datetime.date(2019, 7, 4), 'value': 2088760},
 {'date': datetime.date(2021, 6, 30), 'value': 1920663},
 {'date': datetime.date(2020, 7, 1), 'value': 626516},
 {'date': datetime.date(2019, 7, 3), 'value': 2547889

In [194]:
# turn passengers_per_day into a DataFrame with "date" "value" columns
df = pd.DataFrame(passengers_per_day)
df

,date,value
0,2021-07-05,2160147
1,2020-07-06,755555
2,2019-07-08,2748718
3,2021-07-04,1681896
4,2020-07-05,732123
...,...,...
911,2019-07-10,2515902
912,2020-07-07,641761
913,2019-07-09,2506859
914,2020-07-06,755555


In [195]:
# sort dates from latest to earliest
df.sort_values('date', ascending = False)

,date,value
0,2021-07-05,2160147
3,2021-07-04,1681896
6,2021-07-03,1915017
9,2021-07-02,2196411
12,2021-07-01,2147090
...,...,...
545,2019-01-08,1975947
548,2019-01-07,2150571
551,2019-01-06,2202111
554,2019-01-05,2345103


In [196]:
# delete duplicates

In [197]:
df[df.duplicated(subset = 'date')]

,date,value
914,2020-07-06,755555
915,2019-07-08,2748718


In [198]:
df = df.drop_duplicates(subset = 'date')

In [199]:
df[df.duplicated(subset = 'date')]

,date,value


## 2. Transform the data in two different ways for two different charts
[What's moving average and why are they used? - Dallas FED](https://www.dallasfed.org/research/basics/moving.aspx)

### a. Calculate 7-day moving average

In [211]:
# display the last 7 rows
df.tail(7)

,date,value
907,2019-07-12,2716812
908,2020-07-09,709653
909,2019-07-11,2608209
910,2020-07-08,632498
911,2019-07-10,2515902
912,2020-07-07,641761
913,2019-07-09,2506859


In [212]:

# write a function that takes the current date and 6 previous dates and averages them
def moving_average(row):
    sum_ = 0
    for i in range(0, 7):
        new_date = row['date'] - timedelta(days=i)
        sum_ = sum_ + df[df['date'] == new_date]['value'].item()
    row['7day_average'] = int(sum_ / 7)
    return row

[Read up on pandas' apply method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

In [213]:
# calculate 7-day moving average in a new column and start 7 days in (note: result_type apply)
df_average = df[7:].apply(moving_average, axis=1, result_type='reduce')
df_average

ValueError: can only convert an array of size 1 to a Python scalar

In [207]:
# set the date as the index for matplot
df_average = df_average.set_index('date')

NameError: name 'df_average' is not defined

### b. Group data by weeks

In [ ]:
# create a function to get day of the first day of the week
def weekday_start(row):
    
    return row

In [ ]:
# create a new column that IDs the start date of the week

In [ ]:
# groupby week start turn the groupby object into a dataframe

## 3. Create two charts - one for 7-day moving average and one for week totals
Create a bar chart of the daily values for reference

In [ ]:
# create a bar chart for daily values

### a. 7-day moving average

In [ ]:
# plot a 7-day average line chart

### b. By weekly totals

In [ ]:
# plot as weeks as a line chart